In [1]:
from deepface import DeepFace

In [2]:
import numpy as np
from scipy.spatial.distance import cosine
import json
import cv2
import time

In [3]:
with open("./embeddings.json" , "r") as f:
    embeddings = json.load(f);

In [4]:
models = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
  "OpenFace", 
  "DeepFace", 
  "DeepID", 
  "ArcFace", 
  "Dlib", 
  "SFace",
  "GhostFaceNet"
]

In [9]:

for key1 in embeddings:
    for key2 in embeddings[key1]:
        for index in range(0 , len(embeddings[key1]["embeddings"])):
            # print(embeddings[key1]["embeddings"][index])
            # print(index)



47


This is the code where face detection and face embedding generation were separate task

In [7]:
def recognizer(image , threshold = 0.5):

    start_time = time.time()

    try:
# Detect face
        face = DeepFace.extract_faces(img_path=image, detector_backend='opencv')  # Use default detector
        
        end_time = time.time()  # End timing
        duration = end_time - start_time  # Calculate the time taken
        
        if len(face) > 0:
            print(f"Face detected in {duration:.2f} seconds in image: {image}")
        else:
            print(f"No face detected in image: {image}")
    except Exception as e:
        print(f"An error occurred while processing the image {image}: {str(e)}")
        pass  # Allow the program to continue running

    start_time = time.time()

    try:
        test_embeddings = DeepFace.represent(image, model_name = models[6] , detector_backend="retinaface" , align=True)
        end_time = time.time()  # Record the end time
        duration = end_time - start_time  # Calculate the time taken
        print("time needed to generate the embeddings is " , duration)
    except Exception as e:
        print("Problem with embeddings " , e);

    result = []

    start_time = time.time()

    try:

        for key1 in embeddings:
            for key2 in embeddings[key1]:
                for index in range(0 , len(embeddings[key1]["embeddings"])):
                    stored_embedding = embeddings[key1]["embeddings"][index]
                    distance = cosine(test_embeddings[0]["embedding"], stored_embedding)
                    result.append((key1, distance))
        
        print("finish comparing")
        result = sorted(result, key=lambda x: x[1])
        print("finish sorting")

        end_time = time.time()  # Record the end time
        duration = end_time - start_time  # Calculate the time taken 
        print("finishing the operation " , duration)

            # Return the closest match if within the threshold
        if result[0][1] <= threshold:
            return result[0][0]  # Name of the recognized person
        else:
            return "Unknown"
        

    except Exception as e:
        print("Problem while checking " , e)

    
    

In [9]:
# Initialize the webcam (0 is usually the default camera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open the webcam.")
    exit()

print("Press 'C' to capture an image or 'Q' to quit.")

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Could not read frame from webcam.")
        break

    # Display the resulting frame
    cv2.imshow('Webcam', frame)

    # Wait for a key press
    key = cv2.waitKey(1) & 0xFF

    if key == ord('c'):
        filename = f'./test.jpg'
        cv2.imwrite(filename, frame)
        print(f"Image captured and saved as '{filename}'.")
        face = recognizer(filename)
        print(face)

    elif key == ord('q'):
        print("Exiting...")
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

Now we will combine face detection and embedding generation together

In [25]:
def recognizer(image , threshold = 0.5):

    start_time = time.time()

    try:
# Detect face
        face = DeepFace.extract_faces(img_path=image, detector_backend='opencv')  # Use default detector
        
        end_time = time.time()  # End timing
        duration = end_time - start_time  # Calculate the time taken
        
        if len(face) > 0:
            print(f"Face detected in {duration:.2f} seconds in image: {image}")
            print(len(face))
        else:
            print(f"No face detected in image: {image}")
    except Exception as e:
        print(f"An error occurred while processing the image {image}: {str(e)}")
        pass  # Allow the program to continue running

    start_time = time.time()

    try:
        for i , face_data in enumerate(face):
            cropped_face = face_data['face']
            test_embeddings = DeepFace.represent(img_path=cropped_face, model_name = models[6] , detector_backend="skip" , align=True)
        end_time = time.time()  # Record the end time
        duration = end_time - start_time  # Calculate the time taken
        print("time needed to generate the embeddings is " , duration)
    except Exception as e:
        print("Problem with embeddings " , e);

    result = []

    start_time = time.time()

    try:

        for key1 in embeddings:
            for key2 in embeddings[key1]:
                for index in range(0 , len(embeddings[key1]["embeddings"])):
                    stored_embedding = embeddings[key1]["embeddings"][index]
                    distance = cosine(test_embeddings[0]["embedding"], stored_embedding)
                    result.append((key1, distance))
        
        print("finish comparing")
        result = sorted(result, key=lambda x: x[1])
        print("finish sorting")

        end_time = time.time()  # Record the end time
        duration = end_time - start_time  # Calculate the time taken 
        print("finishing the operation " , duration)

            # Return the closest match if within the threshold
        if result[0][1] <= threshold:
            return result[0][0]  # Name of the recognized person
        else:
            return "Unknown"
        

    except Exception as e:
        print("Problem while checking " , e)

    
    

In [ ]:
# Initialize the webcam (0 is usually the default camera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open the webcam.")
    exit()

print("Press 'C' to capture an image or 'Q' to quit.")

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Could not read frame from webcam.")
        break

    # Display the resulting frame
    cv2.imshow('Webcam', frame)

    # Wait for a key press
    key = cv2.waitKey(1) & 0xFF

    if key == ord('c'):
        filename = f'./test.jpg'
        cv2.imwrite(filename, frame)
        print(f"Image captured and saved as '{filename}'.")
        face = recognizer(filename)
        print(face)

    elif key == ord('q'):
        print("Exiting...")
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

Press 'C' to capture an image or 'Q' to quit.
Image captured and saved as './test.jpg'.
Face detected in 0.13 seconds in image: ./test.jpg
1
time needed to generate the embeddings is  0.3876533508300781
finish comparing
finish sorting
finishing the operation  0.006980419158935547
Ronak
Image captured and saved as './test.jpg'.
Face detected in 0.32 seconds in image: ./test.jpg
1
time needed to generate the embeddings is  0.3177187442779541
finish comparing
finish sorting
finishing the operation  0.005986452102661133
Unknown
Image captured and saved as './test.jpg'.
Face detected in 0.27 seconds in image: ./test.jpg
1
time needed to generate the embeddings is  0.347567081451416
finish comparing
finish sorting
finishing the operation  0.005995035171508789
Unknown
Image captured and saved as './test.jpg'.
Face detected in 0.24 seconds in image: ./test.jpg
1
time needed to generate the embeddings is  0.2966036796569824
finish comparing
finish sorting
finishing the operation  0.006170272827

Till now what will have done include for the one face in the image


Now we are focusing on the mutiple face , embeddings generation and recognition

In [3]:
image_path = "./multiple_faces.jpg"
image = cv2.imread(image_path)

In [5]:
multiple_face_embeddings = DeepFace.represent(image, model_name = models[6] , detector_backend="retinaface" , align=True)

In [7]:
# Display results
for idx, embedding in enumerate(multiple_face_embeddings):
    print(f"Face {idx + 1}:")
    print(f"Embedding Vector: {embedding['embedding']}")
    print(f"Face Detected at Region: {embedding['facial_area']}")

Face 1:
Embedding Vector: [0.0136647280305624, 0.11327403783798218, -0.01616290584206581, 0.33089545369148254, -0.12592439353466034, 0.08943546563386917, -0.19060775637626648, 0.052267711609601974, -0.21408748626708984, -0.17860469222068787, 0.07899901270866394, -0.07535164803266525, -0.5799703598022461, -0.32926928997039795, -0.1120067909359932, 0.11869164556264877, -0.07600345462560654, -0.09064716845750809, -0.16917090117931366, -0.22726872563362122, 0.5282384753227234, 0.16928502917289734, -0.02038239687681198, -0.15225858986377716, -0.10367808490991592, -0.4172073304653168, 0.040314145386219025, -0.08111102133989334, 0.18980735540390015, 0.053889431059360504, -0.32657918334007263, 0.07282137125730515, 0.11802736669778824, -0.18849439918994904, -0.06754346936941147, 0.10532025247812271, -0.5033209919929504, -0.12717309594154358, 0.447416752576828, -0.13044048845767975, 0.08356121927499771, -0.15658314526081085, -0.24963022768497467, -0.07571865618228912, -0.4933120012283325, -0.170

now we are ploting bouding box around the faces

In [8]:
# Draw bounding boxes for each detected face
for result in multiple_face_embeddings:
    facial_area = result['facial_area']
    x, y, w, h = facial_area['x'], facial_area['y'], facial_area['w'], facial_area['h']
    
    # Draw rectangle on the image
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green box with thickness 2

# Display the image with bounding boxes
cv2.imshow("Detected Faces", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Now we will update our logic of recognizing the face which include only one face to now multiple faces

In [12]:
def multiple_face_recognizer(image,threshold=0.5):
    """
    Recognize multiple faces in an image and match them with stored embeddings.

    Parameters:
        image (str): Path to the input image.
        embeddings (dict): Dictionary of stored embeddings.
        models (list): List of model names (ensure the required model is in the list).
        threshold (float): Threshold for cosine similarity to classify a face as recognized.

    Returns:
        results (list): List of recognized names or "Unknown" for each face in the image.
    """
    results = []  # To store recognition results
    start_time = time.time()

    try:
        # Detect faces
        faces = DeepFace.extract_faces(img_path=image, detector_backend='opencv')
        detection_duration = time.time() - start_time
        print(f"Detected {len(faces)} face(s) in {detection_duration:.2f} seconds.")

        # If no faces are detected
        if not faces:
            return ["No face detected"]

        for face_data in faces:
            try:
                # Extract the cropped face image
                face_image = face_data['face']
                facial_area = face_data['facial_area']

                
                # Generate embeddings for the detected face
                embedding_start_time = time.time()
                test_embeddings = DeepFace.represent(
                    img_path=face_image, 
                    model_name=models[6], 
                    detector_backend="skip", 
                    align=True
                )
                embedding_duration = time.time() - embedding_start_time
                print(f"Embedding generated in {embedding_duration:.2f} seconds.")

                # Compare with stored embeddings
                min_distance = float('inf')
                recognized_name = "Unknown"

                for key1 in embeddings:
                    for key2 in embeddings[key1]:
                        for index in range(len(embeddings[key1]["embeddings"])):
                            stored_embedding = embeddings[key1]["embeddings"][index]
                            distance = cosine(test_embeddings[0]["embedding"], stored_embedding)
                            if distance < min_distance:
                                min_distance = distance
                                recognized_name = key1 if distance <= threshold else "Unknown"
                
                # Append the result for the current face
                results.append((recognized_name, min_distance , facial_area))
            
            except Exception as e:
                print(f"Error processing a face: {str(e)}")
                results.append("Error processing face")

        total_duration = time.time() - start_time
        print(f"Processed all faces in {total_duration:.2f} seconds.")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return ["Error in face detection or recognition"]

    return results

In [16]:
# Initialize the webcam (0 is usually the default camera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open the webcam.")
    exit()

print("Press 'C' to capture an image or 'Q' to quit.")

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Could not read frame from webcam.")
        break

    # Display the resulting frame
    cv2.imshow('Webcam', frame)

    # Wait for a key press
    key = cv2.waitKey(1) & 0xFF

    if key == ord('c'):
        filename = f'./test.jpg'
        cv2.imwrite(filename, frame)
        print(f"Image captured and saved as '{filename}'.")
        faces = multiple_face_recognizer(filename)
        print(faces)
        

        for result_data in faces:             
            if type(result_data) != str:
                print("this is string")
                x , y , w ,h = result_data[2]['x'] ,  result_data[2]['y'] ,  result_data[2]['w'] ,  result_data[2]['y']
                cv2.rectangle(frame , (x,y) , (x+w , y+h) , (123,255,0) , 2)
                cv2.putText(frame , f"{result_data[0]}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 12, 0), 1)
            else:
                print("There was no face in the image to detect")
        
        cv2.imshow("faces with recongnized faces" , frame)

    elif key == ord('q'):
        print("Exiting...")
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

Press 'C' to capture an image or 'Q' to quit.
Image captured and saved as './test.jpg'.
Detected 1 face(s) in 0.06 seconds.
Embedding generated in 0.13 seconds.
Processed all faces in 0.19 seconds.
[('Ronak', np.float64(0.16500199678700467), {'x': 224, 'y': 198, 'w': 162, 'h': 162, 'left_eye': None, 'right_eye': None})]
this is string
Exiting...
